<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=f660f5d66b491ac88e41423c981c6dd83de19afb7f24165327d123dce930abf5
  Stored in directory: /home/runner/.cache/pip/wheels/ec/b6/81/d781ff040ee9432384560f29596ee14ca79d70529d4228e50f
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.2
    Uninstalling protobuf-6.33.2:
      Successfully uninstalled protobuf-6.33.2


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-12-16 10:45:40
-------------------
qualified stocks: 86
with latest results: 25
still star stocks: 15
-------------------
Initial Investment:  1.28 C
CY Investment:  1.55 C
Reserve:  66.76 K
Current:  1.41 C
-------------------
Today PnL: -8.34 K (-0.06%)
Current PnL: -26.06 L (-16.83%)
CY Booked + Current PnL: -11.77 L (-7.6%)
-------------------
Total profit:  1.46 L
Total loss:  -27.52 L
-------------------
Total Booked + Current PnL: 15.21 L (11.9%)
Total Booked PnL: 41.27 L (32.29%)
Curr Year Booked PnL: 14.29 L (10.13%)
Prev Year Booked PnL: 26.98 L (21.11%)
Est FTT:  2.32 C
Est FTT PnL: 90.69 L (64.32%)
Deployed:  1.28 C
Current:  1.41 C
CAGR/XIRR %: 5.47%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT','Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'FTT Amt', 'Current P/L', 'Current', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%',  'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
77,TTKPRESTIG,770.00,-0.46,-17.66,21.57,0.09,17898.0,-17802.0,82975.0,92.53,37.0,M-SC,2.34,253.0,-0.99,0.59,9.01,OX40N,NTT,DURABLES
82,VOLTAS,1530.00,-0.13,8.51,10.30,19.69,21431.0,16323.0,208065.0,-2.04,57.0,X-MC,2.52,78.0,0.76,1.48,15.89,XY25,NTT,AC
39,INDIGOPNTS,1408.00,-1.23,-12.76,14.65,0.02,22310.0,-22275.0,152284.0,132.14,51.0,M-SC,14.82,234.0,-1.00,1.09,32.46,OX40N,NTT,PAINTS
50,MASFIN,398.61,-1.41,-1.44,23.83,22.05,23013.0,-1410.0,96570.0,-15.09,56.0,H-SC,6.92,164.0,-0.06,0.69,40.37,XR,ATH,FINANCE
43,ITC,452.00,0.51,-0.90,11.78,10.78,28055.0,-2156.0,238162.0,-41.15,50.0,X-LC,1.92,5.0,-0.08,1.70,4.29,X40,NTT,FMCG


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
61,ROUTE,2227.21,2.35,-46.72,217.27,69.05,154049.0,-62164.0,70902.0,-56.81,53.0,H-SC,22.85,140.0,-0.40,0.51,9.35,SR,ATH,IT
23,DMART,5391.45,2.16,-6.05,37.95,29.60,88992.0,-15102.0,234498.0,-19.86,43.0,X-LC,5.64,19.0,-0.17,1.67,14.82,X40N,ATH,FMCG
44,JCHAC,2282.00,2.03,-33.62,50.63,-0.01,42954.0,-42965.0,84840.0,17269.57,40.0,M-SC,7.30,233.0,-1.00,0.60,8.59,OX40N,NTT,AC
80,VALIANTORG,838.00,1.56,-43.71,209.40,74.15,192834.0,-71516.0,92089.0,-265.94,47.0,H-SC,22.46,149.0,-0.37,0.66,18.04,XR,NTT,CHEMICALS
16,CERA,9475.00,1.55,-31.22,80.13,23.89,96941.0,-54923.0,120980.0,-33.65,34.0,H-SC,10.62,157.0,-0.57,0.86,4.48,OX40N,NTT,CERAMICS


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
57,RAJOOENG,143.10,-1.47,-34.80,114.13,39.61,76273.0,-35670.0,66830.0,-55.84,36.0,H-SC,28.69,135.0,-0.47,0.48,5.29,AR,ATH,MISC
32,HCLTECH,1922.01,-1.41,7.75,15.76,24.73,41081.0,18743.0,260667.0,12.34,59.0,X-LC,6.87,7.0,0.46,1.86,24.16,X40,ATH,IT
50,MASFIN,398.61,-1.41,-1.44,23.83,22.05,23013.0,-1410.0,96570.0,-15.09,56.0,H-SC,6.92,164.0,-0.06,0.69,40.37,XR,ATH,FINANCE
45,JIOFIN,387.00,-1.27,-4.85,30.83,24.48,78428.0,-12977.0,254388.0,-6.45,42.0,X-LC,5.59,36.0,-0.17,1.81,47.45,XY24,BTT,FINANCE
48,KPIGREEN,730.66,-1.23,-14.11,71.09,46.95,76505.0,-17680.0,107617.0,-20.72,45.0,H-SC,7.47,154.0,-0.23,0.77,34.82,MH,ATH,POWER


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
25,FINCABLES,1641.55,1.03,-2.98,114.29,107.91,163722.0,-4393.0,143251.0,-20.96,55.0,M-SC,12.60,216.0,-0.03,1.02,6.81,OX40N,ATH,CABLES
39,INDIGOPNTS,1408.00,-1.23,-12.76,14.65,0.02,22310.0,-22275.0,152284.0,132.14,51.0,M-SC,14.82,234.0,-1.00,1.09,32.46,OX40N,NTT,PAINTS
77,TTKPRESTIG,770.00,-0.46,-17.66,21.57,0.09,17898.0,-17802.0,82975.0,92.53,37.0,M-SC,2.34,253.0,-0.99,0.59,9.01,OX40N,NTT,DURABLES
66,SIS,528.00,0.73,-23.67,58.87,21.26,49892.0,-26283.0,84749.0,2000.00,52.0,H-SC,5.09,166.0,-0.53,0.60,14.50,OX40N,NTT,MISC
47,KANSAINER,340.00,-0.31,-24.69,50.67,13.47,102908.0,-66573.0,203094.0,-69.09,37.0,H-SC,5.18,158.0,-0.65,1.45,3.27,XY24,NTT,PAINTS


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,WIPRO,420.0,-0.71,6.59,61.61,72.27,111278.0,11169.0,180617.0,-8.35,65.0,M-LC,2.46,99.0,0.1,1.29,13.17,XR,NTT,IT


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,WIPRO,420.00,-0.71,6.59,61.61,72.27,111278.0,11169.0,180617.0,-8.35,65.0,M-LC,2.46,99.0,0.10,1.29,13.17,XR,NTT,IT
38,INDIAMART,4810.62,-0.10,-2.70,112.46,106.72,134955.0,-3333.0,120003.0,-51.77,38.0,H-SC,2.95,139.0,-0.02,0.86,20.04,AR,ATH,MISC
25,FINCABLES,1641.55,1.03,-2.98,114.29,107.91,163722.0,-4393.0,143251.0,-20.96,55.0,M-SC,12.60,216.0,-0.03,1.02,6.81,OX40N,ATH,CABLES
50,MASFIN,398.61,-1.41,-1.44,23.83,22.05,23013.0,-1410.0,96570.0,-15.09,56.0,H-SC,6.92,164.0,-0.06,0.69,40.37,XR,ATH,FINANCE
85,ZYDUSLIFE,1286.17,-0.02,-2.84,39.59,35.63,80617.0,-5945.0,203629.0,-16.74,38.0,H-MC,2.91,119.0,-0.07,1.45,13.36,AR,ATH,PHARMA


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
9,BAJAJHFL,181.50,-0.38,-18.25,90.19,55.47,179366.0,-44410.0,198876.0,-27.60,25.0,X-MC,12.93,64.0,-0.25,1.42,1.90,X40N,ATH,FINANCE
53,PAGEIND,51605.07,-0.33,-9.09,40.92,28.11,74925.0,-18310.0,183100.0,-34.13,29.0,X-MC,12.91,55.0,-0.24,1.30,0.00,X40,ATH,APPARELS
75,TMPV,600.00,-0.23,-29.01,73.24,22.98,117448.0,-65538.0,160360.0,-26.74,30.0,X-LC,6.79,3.0,-0.56,1.14,0.86,XY24,NTT,AUTO
11,BATAINDIA,2096.00,-0.92,-38.69,120.52,35.20,95081.0,-49778.0,78892.0,3.39,32.0,X-SC,16.12,92.0,-0.52,0.56,0.21,X40,NTT,FOOTWEAR
51,MEDANTA,1486.00,-0.98,-5.06,30.69,24.08,36988.0,-6428.0,120522.0,-12.71,32.0,X-SC,5.13,89.0,-0.17,0.86,13.56,XY24,NTT,HEALTHCARE


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
15,CAMS,950.0,-0.74,-1.78,26.73,24.48,66121.0,-4488.0,247368.0,-81.32,42.0,X-SC,0.30,86.0,-0.07,1.76,20.94,X40N,NTT,MISC
43,ITC,452.0,0.51,-0.90,11.78,10.78,28055.0,-2156.0,238162.0,-41.15,50.0,X-LC,1.92,5.0,-0.08,1.70,4.29,X40,NTT,FMCG
33,HINDUNILVR,2922.0,0.06,-10.73,27.33,13.67,62720.0,-27573.0,229490.0,-17.44,33.0,X-LC,2.13,14.0,-0.44,1.64,8.34,XY25,NTT,FMCG
36,ICICIPRULI,790.0,-0.03,7.28,21.86,30.73,46200.0,14341.0,211346.0,-17.82,64.0,X-MC,2.32,75.0,0.31,1.51,20.98,X40,ATH,INSURANCE
78,UNITDSPR,1644.0,0.60,6.60,13.29,20.77,33170.0,15445.0,249589.0,-2.63,58.0,X-MC,2.46,62.0,0.47,1.78,13.97,X40N,NTT,BREWERIES


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
53,PAGEIND,51605.07,-0.33,-9.09,40.92,28.11,74925.0,-18310.0,183100.0,-34.13,29.0,X-MC,12.91,55.0,-0.24,1.30,0.00,X40,ATH,APPARELS
3,ACC,3906.00,-0.27,-25.50,120.33,64.15,213321.0,-60671.0,177280.0,-56.31,34.0,X-SC,2.60,82.0,-0.28,1.26,0.06,XY24,BTT,CEMENT
11,BATAINDIA,2096.00,-0.92,-38.69,120.52,35.20,95081.0,-49778.0,78892.0,3.39,32.0,X-SC,16.12,92.0,-0.52,0.56,0.21,X40,NTT,FOOTWEAR
75,TMPV,600.00,-0.23,-29.01,73.24,22.98,117448.0,-65538.0,160360.0,-26.74,30.0,X-LC,6.79,3.0,-0.56,1.14,0.86,XY24,NTT,AUTO
9,BAJAJHFL,181.50,-0.38,-18.25,90.19,55.47,179366.0,-44410.0,198876.0,-27.60,25.0,X-MC,12.93,64.0,-0.25,1.42,1.90,X40N,ATH,FINANCE


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
55,QUESS,424.00,-0.21,-29.14,101.59,42.84,46794.0,-18944.0,46062.0,-53.72,44.0,X-SC,35.51,83.0,-0.40,0.33,4.21,XY24,NTT,MISC
59,RELAXO,1176.00,0.26,-47.51,190.16,52.31,144894.0,-68964.0,76196.0,-44.15,43.0,X-SC,6.90,91.0,-0.48,0.54,1.99,X40N,NTT,FOOTWEAR
11,BATAINDIA,2096.00,-0.92,-38.69,120.52,35.20,95081.0,-49778.0,78892.0,3.39,32.0,X-SC,16.12,92.0,-0.52,0.56,0.21,X40,NTT,FOOTWEAR
51,MEDANTA,1486.00,-0.98,-5.06,30.69,24.08,36988.0,-6428.0,120522.0,-12.71,32.0,X-SC,5.13,89.0,-0.17,0.86,13.56,XY24,NTT,HEALTHCARE
34,HONAUT,58357.33,-0.38,-16.66,71.97,43.32,97692.0,-27132.0,135740.0,-28.43,36.0,X-SC,4.75,90.0,-0.28,0.97,4.22,X40N,ATH,ELECTRICAL


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
73,TCS,4389.97,-0.93,-12.01,37.18,20.71,123743.0,-45416.0,332821.0,-24.27,58.0,X-LC,4.46,1.0,-0.37,2.37,11.21,X40,ATH,IT
41,INFY,1972.00,-1.12,8.16,24.12,34.24,81625.0,25526.0,338414.0,-15.65,58.0,X-LC,3.99,2.0,0.31,2.41,17.12,X40,NTT,IT
75,TMPV,600.00,-0.23,-29.01,73.24,22.98,117448.0,-65538.0,160360.0,-26.74,30.0,X-LC,6.79,3.0,-0.56,1.14,0.86,XY24,NTT,AUTO
81,VBL,671.64,0.81,-2.75,39.50,35.67,121330.0,-8677.0,307165.0,-14.36,59.0,X-LC,4.11,4.0,-0.07,2.19,10.73,X40N,ATH,FMCG
43,ITC,452.00,0.51,-0.90,11.78,10.78,28055.0,-2156.0,238162.0,-41.15,50.0,X-LC,1.92,5.0,-0.08,1.70,4.29,X40,NTT,FMCG


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
40,INDUSINDBK,1800.00,-0.38,-37.39,112.26,32.89,54262.0,-28870.0,48336.0,-697.12,55.0,L-MC,5.98,259.0,-0.53,0.34,33.11,XR,NTT,BANKS
6,ASIANTILES,137.00,-0.62,-18.99,124.26,81.67,94433.0,-17814.0,75996.0,6827.78,38.0,L-SC,12.22,271.0,-0.19,0.54,47.17,XR,NTT,CERAMICS
50,MASFIN,398.61,-1.41,-1.44,23.83,22.05,23013.0,-1410.0,96570.0,-15.09,56.0,H-SC,6.92,164.0,-0.06,0.69,40.37,XR,ATH,FINANCE
13,BSOFT,831.70,-0.38,-21.48,92.03,50.79,101638.0,-30213.0,110440.0,0.89,68.0,H-SC,6.38,151.0,-0.30,0.79,28.59,XR,ATH,IT
39,INDIGOPNTS,1408.00,-1.23,-12.76,14.65,0.02,22310.0,-22275.0,152284.0,132.14,51.0,M-SC,14.82,234.0,-1.00,1.09,32.46,OX40N,NTT,PAINTS


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
13,BSOFT,831.70,-0.38,-21.48,92.03,50.79,101638.0,-30213.0,110440.0,0.89,68.0,H-SC,6.38,151.0,-0.30,0.79,28.59,XR,ATH,IT
84,WIPRO,420.00,-0.71,6.59,61.61,72.27,111278.0,11169.0,180617.0,-8.35,65.0,M-LC,2.46,99.0,0.10,1.29,13.17,XR,NTT,IT
41,INFY,1972.00,-1.12,8.16,24.12,34.24,81625.0,25526.0,338414.0,-15.65,58.0,X-LC,3.99,2.0,0.31,2.41,17.12,X40,NTT,IT
1,ABB,7934.00,-0.33,-3.36,50.62,45.57,127988.0,-8779.0,252840.0,-37.87,58.0,H-MC,4.55,120.0,-0.07,1.80,7.59,AR,NTT,ELECTRICAL
81,VBL,671.64,0.81,-2.75,39.50,35.67,121330.0,-8677.0,307165.0,-14.36,59.0,X-LC,4.11,4.0,-0.07,2.19,10.73,X40N,ATH,FMCG


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,20.25
1,25,44.88
2,50,76.66


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    44.96
MC    30.99
LC    24.05
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     26.74
X40      22.95
X40N     13.07
XR        9.77
XY25      9.10
AR        9.05
OX40N     7.55
SR        1.00
MH        0.77
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
X-MC    24.51
H-SC    23.85
X-LC    20.64
M-SC    11.68
X-SC     8.02
H-MC     4.81
L-SC     1.41
M-MC     1.33
M-LC     1.29
H-LC     1.12
L-LC     1.00
L-MC     0.34
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,17.48,-6.47,40.71
IT,12.97,-15.54,73.64
FINANCE,9.47,-19.82,71.94
MISC,7.27,-27.40,79.51
ELECTRICAL,6.01,-11.21,51.95
PAINTS,5.86,-12.44,29.14
INSURANCE,4.80,-0.77,35.25
PHARMA,4.12,-1.73,33.75
AUTO,2.80,-33.17,79.33


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3170349.0,21
AR,1300879.0,10
XR,1291281.0,13
X40,1002746.0,14
X40N,851171.0,9
OX40N,730937.0,10
XY25,364695.0,6
SR,280470.0,2
MH,76505.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3633593.0,25
M-SC,1419682.0,15
X-MC,1387800.0,16
X-LC,830071.0,11
X-SC,793308.0,8
H-MC,403302.0,3
L-SC,262421.0,3
M-LC,111278.0,1
H-LC,75916.0,1


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1269619.0      6
           AR         890669.0      5
M-SC       XY24       816066.0      6
H-SC       XR         774967.0      7
X-MC       X40        482185.0      7
           XY24       396988.0      3
H-SC       OX40N      341363.0      4
X-LC       X40        333063.0      5
X-MC       X40N       332142.0      4
X-SC       X40N       308707.0      3
M-SC       OX40N      304457.0      5
X-SC       XY24       297103.0      3
H-SC       SR         280470.0      2
X-LC       X40N       210322.0      2
H-MC       AR         208605.0      2
X-LC       XY24       195876.0      2
H-MC       XY24       194697.0      1
X-SC       X40        187498.0      2
L-SC       XR         177304.0      2
X-MC       XY25       176485.0      2
M-SC       XR         173470.0      2
           AR         125689.0      2
M-LC       XR         111278.0      1
X-LC       XY25        90810.0      2
L-SC       OX40N       85117.0      1
H-SC       MH          76505.0      1
H-LC       AR          75916.0      1
M-MC       XY25        58986.0      1
L-MC       XR          54262.0      1
L-LC       XY25        38414.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 58.0 seconds
